In [1]:
!pip install transformers==4.37.2
!pip install flash_attn timm einops
!pip install accelerate
!pip install bitsandbytes


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import transformers; 
print(transformers.__version__)

4.37.2


In [3]:
import accelerate
print(accelerate.__version__)

0.33.0


In [9]:
import math
import numpy as np
import torch
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)



def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    num_layers = {
        'InternVL2-1B': 24, 'InternVL2-2B': 24, 'InternVL2-4B': 32, 'InternVL2-8B': 32,
        'InternVL2-26B': 48, 'InternVL2-40B': 60, 'InternVL2-Llama3-76B': 80}[model_name]
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

# If you set `load_in_8bit=True`, you will need two 80GB GPUs.
# If you set `load_in_8bit=False`, you will need at least three 80GB GPUs.
path = 'OpenGVLab/InternVL2-Llama3-76B'
cache_dir = './workspace'
device_map = split_model('InternVL2-Llama3-76B')
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    cache_dir=cache_dir,
    device_map=device_map).eval()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)


model-00032-of-00032.safetensors:  50%|#####     | 1.94G/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/32 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# set the max number of tiles in `max_num`
generation_config = dict(max_new_tokens=1024, do_sample=False)

# pure-text conversation (纯文本对话)
question = 'Hello, who are you?'
response, history = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

# question = 'Can you tell me a story?'
# response, history = model.chat(tokenizer, None, question, generation_config, history=history, return_history=True)
# print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


User: Hello, who are you?
Assistant: I am an AI assistant whose name is InternVL, developed jointly by Shanghai AI Lab and SenseTime.


In [11]:
# pure-text conversation (纯文本对话)
question = 'give a list of u.s presidents?'
response, history = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:128003 for open-end generation.


User: give a list of u.s presidents?
Assistant: Here is a list of U.S. Presidents in chronological order:

1. George Washington (1789-1797)
2. John Adams (1797-1801)
3. Thomas Jefferson (1801-1809)
4. James Madison (1809-1817)
5. James Monroe (1817-1825)
6. John Quincy Adams (1825-1829)
7. Andrew Jackson (1829-1837)
8. Martin Van Buren (1837-1841)
9. William Henry Harrison (1841)
10. John Tyler (1841-1845)
11. James K. Polk (1845-1849)
12. Zachary Taylor (1849-1850)
13. Millard Fillmore (1850-1853)
14. Franklin Pierce (1853-1857)
15. James Buchanan (1857-1861)
16. Abraham Lincoln (1861-1865)
17. Andrew Johnson (1865)
18. Ulysses S. Grant (1869-1877)
19. Rutherford B. Hayes (1877-1881)
20. James A. Garfield (1881)
21. Chester A. Arthur (1881-1885)
22. Grover Cleveland (1885-1889)
23. Benjamin Harrison (1889-1893)
24. Grover Cleveland (1893-1897)
25. William McKinley (1897-1901)
26. Theodore Roosevelt (1901-1909)
27. William Howard Taft (1909-1913)
28. Woodrow Wilson (1913-1921)
29. Warr